In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.

    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.

    Note: The layer has been tested with Keras 2.0.6

    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [13]:

train_ling = pd.read_csv("../data/" + "preprocessed/train_ling.csv")

EMBEDDING_FILE = '../../../embeddings/glove.42B.300d.txt'

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../submissions/sample_submission.csv')

X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values

meta_features = [
#     'count_sent', 'count_word', 'count_unique_word', 'count_letters',
#        'count_punctuations',
#     'count_words_upper',
#     'count_words_title',
#        'count_stopwords', 'mean_word_len', 'word_unique_percent',
#        'punct_percent'
#     ,
#                  'count_swear_words'
]

X_meta_features = train_ling[meta_features]
from sklearn.preprocessing import minmax_scale
X_meta_features = minmax_scale(X_meta_features)

def build_input_data(sentences, labels, vocabulary):
    x = np.array([[embeddings_index[vocabulary_inv[vocabulary['word']]] if word in vocabulary.keys() else len(vocabulary) - 1 for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]

In [4]:

max_features = 30000
maxlen = 100
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)



In [5]:

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [6]:


class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))


In [14]:
import keras

def test_model():
    inp = Input(shape=(maxlen, ))
    inp_meta = Input(shape=(len(meta_features),))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    lstm1 = Bidirectional(CuDNNLSTM(80, return_sequences=True))(x)
#     meta_dense = Dense(64)(inp_meta)
    flatten = Flatten()(lstm1)
    conc = concatenate([inp_meta, flatten])
#     dense = Dense(7)(conc)
#     lstm2 = Bidirectional(CuDNNLSTM(80, return_sequences=True))(dense)
    output = Dense(units=6, activation='sigmoid')(conc)
    model = Model(inputs=[inp, inp_meta], outputs=output)
    
    model.compile(loss='binary_crossentropy',optimizer='adam',
#                   optimizer=Adam(lr = 0.001, 
# #                                  decay=0.2
#                                 ),
                  metrics=['accuracy'])
    return model


def ensemble(model_f, num_ensembles, input_length):
    models = [model_f() for i in range(0,num_ensembles)]
    ensemble_input = Input(shape=(input_length,))
    ensemble_meta_input = Input(shape=(len(meta_features),))
    averaged = average([model([ensemble_input, ensemble_meta_input]) for model in models])
    ensemble = Model(inputs=[ensemble_input, ensemble_meta_input], outputs=[averaged])
    ensemble.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])    
    return models, ensemble


def simple_lstm_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    lstm1 = Bidirectional(CuDNNLSTM(80, return_sequences=True))(x )
#     lstm2 = Bidirectional(CuDNNLSTM(20, return_sequences=True))(lstm1)
    avg_pool = GlobalAveragePooling1D()(lstm1)
    max_pool = GlobalMaxPooling1D()(lstm1)
    conc = concatenate([avg_pool, max_pool])
#     dense = Dense(128)(conc)
    output = Dense(units=6, activation='sigmoid')(conc)
    
    model = Model(inputs=inp, outputs=output)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model_func = test_model
ensemble_it = False
num_ensembles = 7
train_ensemble_jointly_after = True
epochs = 3
batch_size = 128
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
X_train_meta, X_val_meta, y_tra2,y_val2 = train_test_split(X_meta_features, y_train, train_size=0.95, random_state=233)
    
if ensemble_it:
    models, ensemble_model = ensemble(model_func, num_ensembles, maxlen)   
    for model in models:
        callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5),
        keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto'),
        RocAucEvaluation(validation_data=([X_val, X_val_meta], y_val), interval=1)
        ]

        hist = model.fit([X_tra, X_train_meta], y_tra, batch_size=batch_size, epochs=epochs, validation_data=([X_val, X_val_meta], y_val),
                         callbacks=callbacks, verbose=2)
    y_val_pred = ensemble_model.predict([X_val, X_val_meta], batch_size=batch_size)
    score = roc_auc_score(y_val, y_val_pred)
    print(score)

else:
    model = model_func()
    callbacks=[
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5),
    keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto'),
    RocAucEvaluation(validation_data=([X_val, X_val_meta], y_val), interval=1)
    ]
    hist = model.fit([X_tra, X_train_meta], y_tra, batch_size=batch_size, epochs=epochs, validation_data=([X_val, X_val_meta], y_val),
                     callbacks=callbacks, verbose=2)
    y_val_pred = model.predict([X_val, X_val_meta], batch_size=batch_size)
    score = roc_auc_score(y_val, y_val_pred)
    print(score)

if ensemble_it and train_ensemble_jointly_after:
    epochs=1
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5),
        keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto'),
        RocAucEvaluation(validation_data=([X_val, X_val_meta], y_val), interval=1)
        ]
    hist = ensemble_model.fit([X_tra, X_train_meta], y_tra, batch_size=batch_size, epochs=epochs, validation_data=([X_val, X_val_meta], y_val),
                     callbacks=[callbacks], verbose=2)
    y_val_pred = model.predict([X_val, X_val_meta], batch_size=2)
    score = roc_auc_score(y_val, y_val_pred)
    print(score) 

Train on 151592 samples, validate on 7979 samples
Epoch 1/3
 - 42s - loss: 0.0550 - acc: 0.9804 - val_loss: 0.0512 - val_acc: 0.9810

 ROC-AUC - epoch: 1 - score: 0.984961 

Epoch 2/3
 - 42s - loss: 0.0392 - acc: 0.9850 - val_loss: 0.0485 - val_acc: 0.9818

 ROC-AUC - epoch: 2 - score: 0.982992 

Epoch 3/3
 - 41s - loss: 0.0305 - acc: 0.9882 - val_loss: 0.0533 - val_acc: 0.9811

 ROC-AUC - epoch: 3 - score: 0.980961 

0.9809607034655509


In [8]:
# callbacks=[
# keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5),
# keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto'),
# RocAucEvaluation(validation_data=([X_val, X_val_meta], y_val), interval=1)
# ]
# hist = model.fit([X_tra, X_train_meta], y_tra, batch_size=batch_size, epochs=100, validation_data=([X_val, X_val_meta], y_val),
#                  callbacks=callbacks, verbose=2)
# y_val_pred = model.predict([X_val, X_val_meta], batch_size=batch_size)
# score = roc_auc_score(y_val, y_val_pred)
# print(score)

In [9]:
model.layers[4].get_weights()[0].mean()
# print(len(meta_features))

IndexError: list index out of range

In [ ]:
y_pred = model.predict(x_test, batch_size=128)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)

In [ ]:
y_val_pred = ensemble_model.predict(X_val, batch_size=128)
score = roc_auc_score(y_val, y_val_pred)
print(score)

In [ ]:
print(x_test.shape, y_pred.shape, y_val.shape)